# 本セクションの目次
1. Spark Structured Streamingとは？
2. メッセージキューとの組み合わせは？
3. 他のストリーミングツール

# Spark Structured Streamingとは？
Spark Structured Streamingとは
「Spark Streaming」はApache Sparkをストリーミング用途で利用する場合に使われる呼び方です。  
ですので、Sparkがあればバッチ処理からストリーミング処理まで完結します。ビッグデータの処理において、使い勝手の良い技術の一つでしょう。  

## PySparkのバッチとはどのように違う？
あくまでSparkの中の機能の一つなので、基本的な使い方は変わりません。  
そのためデータフレームの操作やSparkSQLの操作はバッチと変わりません。

しかし、ストリーミング用の関数を使う必要があったりすることと、登場人物が多い(Kafka（メッセージキュー）などとの連携がある)のでプログラムが複雑になりやすいです。

# Kafkaとの組み合わせはどんな所に使われる？
Kafkaとの組み合わせでは、Spark Streamingはパブリッシャーになることもできますしコンシューマーになることも可能です。

##  kafkaに送信されるデータの形
kafkaに送信するデータはキー、バリュー型です。

```
root
 |-- key: binary (nullable = true)   
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

```


## コマンドラインがパブリッシャーの例とコンシューマーの例

cmd -> kafka <- spark streamingという流れになります。

```
 /home/pyspark/kafka/bin/kafka-console-producer.sh --topic pyspark-topic1 --bootstrap-server kafka:9092 \
 --property parse.key=true --property key.separator=,

```

ーーーーkafkaがデータを保持する


```
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "pyspark-topic1") \
  .load()
```

## Spark Streamingがパブリッシャーの例とコンシューマーの例

cmd -> kafka <- spark streaming -> kafkaという流れになります。

```
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "pyspark-topic1") \
  .load()
```

Streamingがkafkaに対して書き込みを行う。別トピックにする。
```
ds = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("topic", "pyspark-topic2") \
  .option("checkpointLocation","/tmp/kafka") \
  .queryName("aggregates") \
  .start()
```

ーーーーkafka(topic 2)がデータを保持する




# 他のストリーミングツール
Spark Streaming以外でストリーミング処理(プログラム可能なもの)で使われるツールを紹介します。

- Apache Beam

特に大きな違いはないのですが、GoogleであればGoogle Dataflowというマネージドサービスが出ています。  
少し言葉の定義などに癖があるのですが、Sparkを理解していればBeamの理解は簡単なはずです。  

Apache BeamはPythonやJava、Goなどに対応しています。